In [ ]:
#!pip install -U sparkmagic
!pip install -U langchain-google-community\[gmail\] langchain langchain-community langchain-openai

In [ ]:
!pip install -qU  google-api-python-client google-auth-oauthlib google-auth-httplib2

In [26]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [41]:

import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = "pr-new-collard-32"
os.environ["GOOGLE_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"



In [42]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_google_credentials,
    build_gmail_service
)
from langchain_google_community import GmailToolkit

In [43]:
import webbrowser

webbrowser.register('chrome', None, webbrowser.GenericBrowser('/usr/bin/google-chrome'))

In [44]:
from google_auth_oauthlib.flow import Flow
from google.oauth2.credentials import Credentials
import os

# Define your scopes and client secrets file
scopes = ['https://mail.google.com/']
client_secrets_file = 'client_secret_704484678251-nlq3n7rccoua367fbkovu83rskol0iqc.apps.googleusercontent.com.json'
token_file = 'token.json'

# Create the OAuth flow manually
flow = Flow.from_client_secrets_file(
    client_secrets_file=client_secrets_file,
    scopes=scopes,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'  # This is the out-of-band redirect URI
)

# Generate authorization URL
auth_url, _ = flow.authorization_url(prompt='consent')

print("Please go to this URL and authorize the application:")
print(auth_url)

# Ask user to paste the authorization code
code = input("Enter the authorization code here: ")
flow.fetch_token(code=code)

# Get credentials
creds = flow.credentials

# Save credentials for future use
with open(token_file, 'w') as token:
    token.write(creds.to_json())



Please go to this URL and authorize the application:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=704484678251-nlq3n7rccoua367fbkovu83rskol0iqc.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fmail.google.com%2F&state=sqomvivpqY6X8NHHjCU3eyJlQzfEel&prompt=consent&access_type=offline
Enter the authorization code here: 4/1Ab32j93UiCNSVpyiCiMOXtLzLT96wXPxyM0IfMTamZ51MKykdleZ7CbeDGE


In [45]:
api_resource = build_gmail_service(credentials=creds)
toolkit = GmailToolkit(api_resource=api_resource)

In [46]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x7a9d687f4a40>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7a9d687f4a40>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x7a9d687f4a40>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7a9d687f4a40>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x7a9d687f4a40>)]

In [47]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="openai/gpt-5",
    temperature=0.7
)


In [48]:
SYSTEM_PROMPT = """
You are an intelligent email assistant designed to help users manage their email communications.
You can create drafts, send emails, and delete emails using the provided tools.
Agent should not stop without completing the operation
Follow these rules:

1. Drafting Emails:
   - When the user provides email content (recipients, subject, body),
     use the draft creation tool and create a draft email in gmail box.

2. Sending Emails:
   - You can send emails directly or from saved drafts.
   - Notify the user once the email is successfully sent.
   - don't stop without sending email.
   - email should be send from logged in mail box

3. Search emails:
   - You can search email from my inbox and send me the details.

General Behavior:
- Be polite, clear, and concise.
- Log actions where appropriate for traceability.
- Don't stop until the operation specified is completed.
- create the email in the mail box
- Search email and tool calling should happen.
- Do't add your pretrained details
"""



In [49]:
from langchain.agents import create_agent


agent_executor = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)

In [50]:
example_query = "Create a draft email to nitya.cse.hit@gmail.com thanking them for coffee."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Create a draft email to nitya.cse.hit@gmail.com thanking them for coffee.
================================== Ai Message ==================================
Tool Calls:
  create_gmail_draft (call_8daUaUH02Kn4CXZyHEjPJ8yz)
 Call ID: call_8daUaUH02Kn4CXZyHEjPJ8yz
  Args:
    message: Hi Nitya,

Thank you so much for the coffee—really appreciated it. I enjoyed our conversation and the chance to connect. Let’s keep in touch.

Best regards,
[Your Name]
    to: ['nitya.cse.hit@gmail.com']
    subject: Thank you for the coffee
    cc: None
    bcc: None
================================= Tool Message =================================
Name: create_gmail_draft

Draft created. Draft Id: r-7986974433448203221
================================== Ai Message ==================================

Draft created successfully.
Subject: Thank you for the coffee
To: nitya.cse.hit@gmail.com

Hi Nitya,

Thank you so much for the cof